In [1]:
def load_namespace():
    import sys
    sys.path.insert(1,f'/wsu/home/gy/gy40/gy4065/hm_jetscapeml_source')#WSU Grid
    sys.path.insert(1,'/content/drive/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source')#Colab GDrive v1
    sys.path.insert(1,'/content/drive/MyDrive/Projects/110_JetscapeMl/hm_jetscapeml_source')#Colab GDrive v2
    sys.path.insert(1,f'/mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source')#wsl gdrive
    sys.path.insert(1,'G:\\My Drive\\Projects\\110_JetscapeMl\\hm_jetscapeml_source') #Windows GDrive
    sys.path.insert(1,'/home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/') #office tower
    
load_namespace()

In [2]:
# Loading/Preparing Environment for simulation
from jet_ml.config import Config
folds=5
epochs=2
dataset_size=1000 #10800000 #1000000
model_name="res_net"
simulation_name=f"alpha_s_{model_name}_{folds}_fold_{epochs}_epoch_{int(dataset_size/1000)}k_dataset_size"

config=Config(simulation_name=simulation_name)
config.__str__()

2024-09-17 23:07:15.685595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 23:07:15.709135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 23:07:15.716980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 23:07:15.737738: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 23:07:17.617969: W tensorflow/compiler/tf2

Directory /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size already exists.
Directory /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/reports/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size already exists.
Directory /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/reports/figures/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size already exists.


I0000 00:00:1726628840.007128    4908 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726628840.063622    4908 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726628840.064311    4908 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


'Project Root: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source\nData Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/data\nModels Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models\nReports Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/reports\nFigures Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/reports/figures\nSimulation Models Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size\nSimulation Reports Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/reports/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size\nSimulation Figures Directory: /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/reports/figures/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size\nEnvironment Details:\n  TensorFlow Version: 2.17.0\n  Keras Version: 3.5.0\n  Python Version: 3.12.5 | packaged by conda

In [8]:
import tensorflow as tf

# Disable GPU by setting visible devices to only CPU
tf.config.set_visible_devices([], 'GPU')

# Optionally, check if GPUs are available (should show an empty list)
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from IPython.display import display
# with tf.device("CPU"):
import jet_ml.classifiers.alpha_s.preprocess_dataset as pred
(x,y_raw,y_df)=pred.preprocess_dataset_for_alpha_s(dataset_size)
y_classes=y_df.columns
y=y_df.values
display("y_classes: ",y_classes)
display("y: ",y[:5])
display("y_raw: ",y_raw[:5])

In [4]:
def is_normalized(x):
    import numpy as np
    return np.max(x) <= 1

def convert_to_rgb(x):
    import numpy as np
    import tensorflow as tf
    # Convert grayscale to RGB (if needed)
    x_rgb = np.concatenate([x] * 3, axis=-1)  # Convert to RGB
    return x_rgb

def resize_images(x,width=32,height=32,device="/CPU:0"):
    import numpy as np
    import tensorflow as tf
    # Resize images to the target size
    TARGET_WIDTH = width
    TARGET_HEIGHT = height
    # with tf.device('/CPU:0'):
    x_resized = np.array([tf.image.resize(image, [TARGET_HEIGHT, TARGET_WIDTH]) for image in x])
    return x_resized
def resize_y(y):
    import numpy as np
    # Optionally, you might want to add an extra dimension if needed
    y_resized = np.expand_dims(y, axis=2)  # Shape will be (1, 3, 1000)
    return y_resized

In [ ]:
#implementing the preprocess_dataset_for_resnet function
def preprocess_dataset_for_resnet(x,y,width=32,height=32):
    display("x.shape {0}".format(x.shape))
    display("y.shape {0}".format(y.shape))
    
    display("Data is normalized: {0}".format(is_normalized(x)))
    
    x_rgb=convert_to_rgb(x)
    display("x_rgb.shape {0}".format(x_rgb.shape))
    
    x_resized=resize_images(x_rgb,width,height)
    display("x_resized.shape {0}".format(x_resized.shape))
    
    y_resized=resize_y(y)
    display("y_resized.shape {0}".format(y_resized.shape))
    return x_resized,y_resized

WIDTH = 256
HEIGHT = 256
x_resized,y_resized= preprocess_dataset_for_resnet(x,y,WIDTH,HEIGHT)

In [ ]:
import numpy as np
#checking data is normalize
print("checking data is normalize")
x_max=np.max(x)
display(x_max)
display(x.shape)
display(y.shape)

In [7]:
WIDTH = 256
HEIGHT = 256

In [8]:

import numpy as np
import tensorflow as tf

# Convert grayscale to RGB (if needed)
x_rgb = np.concatenate([x] * 3, axis=-1)  # Convert to RGB

# Resize images to the target size
TARGET_WIDTH = 256
TARGET_HEIGHT = 256
with tf.device("GPU"):
    x_resized = np.array([tf.image.resize(image, [HEIGHT, WIDTH]) for image in x_rgb])


In [ ]:
# Transpose and then rearrange the dimensions
y_transposed = np.transpose(y)  # Shape will be (3, 1000)

# y_transposed is now (3, 1000)
print("Transposed shape:", y_transposed.shape)

# Optionally, you might want to add an extra dimension if needed
y_resized = np.expand_dims(y, axis=2)  # Shape will be (1, 3, 1000)

# Check the final shape
print("Final shape:", y_resized.shape)  # Should be (1, 3, 1000)

In [ ]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(x) * TRAIN_PCT)

x_df_train_cut = x_resized[0:TRAIN_CUT]
x_df_validate_cut = x_resized[TRAIN_CUT:]

y_df_train_cut = y_resized[0:TRAIN_CUT]
y_df_validate_cut = y_resized[TRAIN_CUT:]


print(f"Training size: {len(x_df_train_cut)}")
print(f"Validate size: {len(x_df_validate_cut)}")

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define ImageDataGenerator
training_datagen = ImageDataGenerator(
    horizontal_flip=True,
    fill_mode='nearest'
)


# Keeping the training batch size small 
# USUALLY increases performance
train_batch_size = 32

# Create the data generator
train_generator = training_datagen.flow(
    x_df_train_cut,
    y_df_train_cut,
    batch_size=train_batch_size,
    shuffle=True,
    seed=42  # For reproducibility
)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
validation_datagen = ImageDataGenerator()

# Make the validation batch size as large as you 
# have memory for
validation_batch_size = 16

val_generator = validation_datagen.flow(
        x_df_validate_cut,
        y_df_validate_cut,
        batch_size=validation_batch_size,
        )

In [19]:
def build_model(input_tensor,num_classes,activation='softmax'):
    from tensorflow.keras.applications.resnet50 import ResNet50
    base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

    from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    from tensorflow.keras.models import Model
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    model=Model(inputs=base_model.input,outputs=Dense(num_classes,activation=activation)(x),name='ResNet50')
    return model
with tf.device('/CPU:0'):#/GPU:0
    from tensorflow.keras.layers import Input
    input_tensor = Input(shape=(HEIGHT, WIDTH, 3))
    model=build_model(input_tensor,num_classes=3,activation='softmax')
    # model.summary()
    import tensorflow as tf

    from jet_ml.models.helpers import compile_model
    model=compile_model(model)

In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input
import tensorflow as tf
# Enable logging of device placement
tf.debugging.set_log_device_placement(True)

input_tensor = Input(shape=(HEIGHT, WIDTH, 3))
base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

In [23]:

with tf.device('/CPU:0'):
    from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    from tensorflow.keras.models import Model
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    model=Model(inputs=base_model.input,outputs=Dense(y_classes.size,activation='softmax')(x),name='ResNet50')

In [ ]:
model.summary()

In [24]:
with tf.device('/CPU:0'):
    from tensorflow.keras.callbacks import EarlyStopping
    # Important, calculate a valid step size for the validation dataset
    STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import keras
def train_model(model,train_generator, val_generator, epochs,
                monitor=None,
                fold=None,
                verbose=1):
    keras.backend.clear_session()
    from jet_ml.models.helpers import get_best_model_filename
    best_model_filename=get_best_model_filename(model.name,fold=fold)
    
    from jet_ml.models.helpers import get_callbacks
    callbacks = get_callbacks(monitor=monitor,
                              model_checkpoint_best_model_filename=best_model_filename)

    import time
    start_time=time.time()

    STEP_PER_EPOCH=train_generator.n//train_generator.batch_size 
    display(STEP_PER_EPOCH)

    # Important, calculate a valid step size for the validation dataset
    STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
    display(STEP_SIZE_VALID)
    

    history=model.fit(train_generator, 
                    epochs=epochs, steps_per_epoch=STEP_PER_EPOCH, 
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=callbacks,
                    verbose = verbose)

    from jet_ml.models.helpers import extract_stopped_epoch
    stoppped_epoch=extract_stopped_epoch(callbacks=callbacks)
    elapsed_time=time.time()-start_time
    import jet_ml.helpers as helpers
    print("Elpased time: {}".format(helpers.hms_string(elapsed_time)))
    return model, history, elapsed_time,stoppped_epoch

epochs=2
monitor='val_loss' #'val_accuracy' or 'val_loss'
import tensorflow as tf
with tf.device('/CPU:0'):#/GPU:0
    train_model(model,train_generator,val_generator,epochs=epochs,monitor=monitor)



In [ ]:
with tf.device('/CPU:0'):
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=50, verbose=1, mode='auto',
        restore_best_weights=True)
        history=model.fit(train_generator, epochs=5, steps_per_epoch=len(x_resized) // train_batch_size, 
          validation_data=val_generator,callbacks=[monitor],
          verbose = 1,validation_steps=STEP_SIZE_VALID)

In [ ]:
# fold, shuffle, x, y_raw
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
k_fold=StratifiedKFold(folds,shuffle=False)

out_of_sample_y=[]
out_of_sample_pred=[]
fold=0
folds_accuracy=[]
epochs_needed = []
times_taken=[]
#Must specify y StratifiedKFold for classification
for train,test in k_fold.split(x,y_raw):
    fold+=1
    print(f"Fold #{fold}")

    x_train=x[train]
    y_train=y[train]
    x_test=x[test]
    y_test=y[test]

    from jet_ml.models import vgg16_net
    input_shape=x[0].shape
    output_shape=y.shape[1]
    activation='softmax'
    # HIDE OUTPUT
    from tensorflow.keras.applications import MobileNet
    model = MobileNet(weights='imagenet',include_top=True)
    model.summary()
    model=vgg16_net.build_model(input_shape,output_shape,activation)
    model=vgg16_net.compile_model(model)


    batch_size=128
    # batch_size=256 #original patch size applies for eloss classification
    monitor='val_loss' #'val_accuracy' or 'val_loss'
    
    model, history,elapsed_time,stopped_epoch=vgg16_net.train_model(model,
                                        x_train,y_train, x_test,y_test, 
                                        epochs, batch_size, monitor,
                                        fold=fold)

    from jet_ml.evaluation import get_accuracy
    pred, score=get_accuracy(model,x_test=x_test,y_test=y_test)
    
    folds_accuracy.append(score)
    times_taken.append(elapsed_time)    
    epochs_needed.append(epochs)

    out_of_sample_y.append(y_test)
    out_of_sample_pred.append(pred)
    print(f"Fold score (accuracy): {score}")

    from jet_ml.evaluation import save_training_history
    save_training_history(history=history,fold=fold)

    from jet_ml.evaluation import plot_training_history
    plot_training_history(history=history,fold=fold)

from jet_ml.evaluation import save_training_stats
save_training_stats(accuracies=folds_accuracy,
                    epochs_needed=epochs_needed,
                    times_taken=times_taken)

# # Build the oos prediction list and calculate the error.
out_of_sample_y=np.concatenate(out_of_sample_y)
out_of_sample_pred=np.concatenate(out_of_sample_pred)

out_of_sample_y_compare=np.argmax(out_of_sample_y,axis=1)# For accuracy and confusion matrix calculation